# Ligands targeting multiple kinase groups

Extract from the pooled Karaman-Davis profiling dataset ligands that target mulitple kinase groups. We can treat those as "unexpected off-targets".

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display, HTML, Markdown
import pandas as pd
from opencadd.databases import klifs

from src import data

In [3]:
klifs_session = klifs.setup_remote()

## Load profiling data

In [4]:
profiling_data = data.profiling.load("karaman-davis")
profiling_data.index.name = "kinase.name"

No measurements                                                                 20611
One measurement                                                                  6296
Two identical measurements                                                       1291
Two measurements <= or > cutoff 100; keep lower value                              30
One measurement <=, one > cutoff 100 but difference <= 100; keep lower value        2
One measurement <=, one > cutoff 100 but difference > 100; remove values           10
dtype: int64


## Add kinase groups to kinases in profiling data

In [5]:
kinases = klifs_session.kinases.by_kinase_name(profiling_data.index.to_list())
kinases = kinases[kinases["species.klifs"] == "Human"]
kinases.shape

(352, 11)

In [6]:
def fetch_kinase_names(klifs_name, gene_name, profiling_names):
    """
    Kinase names used in profiling dataset can be part of
    KLIFS "kinase.klifs_name" or "kinase.gene_name" columns.
    Add new column to KLIFS data that maps to profiling dataset names.
    """

    if klifs_name == gene_name:
        return klifs_name
    else:
        if klifs_name in profiling_names and gene_name in profiling_names:
            raise KeyError(
                f"One KLIFS kinase ({klifs_name}, {gene_name}) mapped to two profiling kinases."
            )
        elif klifs_name in profiling_names:
            return klifs_name
        elif gene_name in profiling_names:
            return gene_name
        else:
            return None

In [7]:
# Iterate through KLIFS data and add column with kinase name as used in profiling data!
kinases["kinase.name"] = kinases.apply(
    lambda x: fetch_kinase_names(
        x["kinase.klifs_name"], x["kinase.gene_name"], profiling_data.index
    ),
    axis=1,
)

In [8]:
# Add kinase family and group to profiling data (as index!)
profiling_data = pd.merge(
    kinases[["kinase.name", "kinase.family", "kinase.group"]], profiling_data.reset_index()
).set_index(["kinase.name", "kinase.family", "kinase.group"])
profiling_data.head()

,,,Staurosporine,JNJ-7706621,MLN-8054,PKC-412,SU-14813,SB-202190,CP-724714,VX-680/MK-0457,Dasatinib,Roscovitine/CYC-202,...,PD-173955,Pazopanib,AST-487,GDC-0941,TG-100-115,TAE-684,SKI-606,BIBW-2992,AZD-6244/ARRY-886,AZD-2171
kinase.name,kinase.family,kinase.group,,,,,,,,,,,,,,,,,,,,,
DYRK1A,DYRK,CMGC,4.0,NaN,NaN,100.0,260.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,670.0,2900.0,NaN,970.0,NaN,NaN
GPRK4,GRK,AGC,37.0,NaN,NaN,110.0,370.0,NaN,NaN,400.0,NaN,NaN,...,NaN,NaN,NaN,NaN,6200.0,3500.0,4900.0,NaN,NaN,NaN
LATS2,NDR,AGC,17.0,NaN,NaN,2200.0,430.0,1700.0,NaN,6500.0,NaN,NaN,...,NaN,NaN,3000.0,NaN,1800.0,2000.0,NaN,NaN,NaN,NaN
PKACb,PKA,AGC,29.0,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,260.0,NaN,NaN,4100.0,NaN,NaN,NaN,NaN
PKCa,PKC,AGC,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Report ligands targeting multiple kinase groups

In [9]:
def bioactivities_against_multiple_kinase_groups(bioactivities, activity_cutoff):
    """
    Filter input bioactivities for bioactivities below given cutoff.
    Return bioactivities only if measured against multiple kinase groups.
    """

    bioactivities = bioactivities.sort_values()
    bioactivities = bioactivities[bioactivities <= activity_cutoff]

    kinase_groups = bioactivities.index.get_level_values(2).unique()
    if len(kinase_groups) > 1:
        bioactivities.name = "bioactivity"
        return bioactivities
    else:
        return None


def get_pkidb_ligand_info(ligand_name):
    """
    Get brand name for ligand from PKIDB if available.
    """

    pkidb = data.ligands.pkidb()
    ligand_pkidb = pkidb[pkidb.apply(lambda x: ligand_name.upper() in x["Synonyms"], axis=1)]

    if len(ligand_pkidb) == 0:
        return None
    elif len(ligand_pkidb) == 1:
        ligand_name_pkidb = ligand_pkidb["BrandName"].iloc[0]
        ligand_type = ligand_pkidb["Type"].iloc[0]
        return f"{ligand_name_pkidb} | Type {ligand_type}"
    else:
        raise ValueError(f"Multiple entries in PKIDB for {ligand_name}.")

In [10]:
ACTIVITY_CUTOFF = 5

In [11]:
for ligand_name, bioactivities in profiling_data.iteritems():

    # Omit highly promiscuous ligands
    if ligand_name in ["Staurosporine"]:
        continue

    bioactivities = bioactivities_against_multiple_kinase_groups(bioactivities, ACTIVITY_CUTOFF)

    if bioactivities is not None:

        # Add Markdown section of ligand name (and brand name if applicable)
        ligand_pkidb_info = get_pkidb_ligand_info(ligand_name)
        if ligand_pkidb_info is not None:
            display(Markdown(f"### {ligand_name} (PKIDB: {ligand_pkidb_info})"))
        else:
            display(Markdown(f"### {ligand_name}"))

        # Add bioactivites as DataFrame
        display(HTML(bioactivities.reset_index().to_html()))

### SU-14813

,kinase.name,kinase.family,kinase.group,bioactivity
0,PDGFRb,PDGFR,TK,0.29
1,FLT3,PDGFR,TK,0.54
2,KIT,PDGFR,TK,0.68
3,PDGFRa,PDGFR,TK,1.10
4,KDR,VEGFR,TK,2.30
5,FMS,PDGFR,TK,3.60
6,HUNK,CAMKL,CAMK,3.70
7,FLT1,VEGFR,TK,4.70


### VX-680/MK-0457

,kinase.name,kinase.family,kinase.group,bioactivity
0,AurA,Aur,Other,3.9
1,ABL2,Abl,TK,4.0


### Dasatinib (PKIDB: Dasatinib;Sprycel | Type 1)

...

,kinase.name,kinase.family,kinase.group,bioactivity
0,TIE1,Tie,TK,0.29
1,CDKL2,CDKL,CMGC,0.52
2,DDR1,DDR,TK,0.69
3,FLT3,PDGFR,TK,0.79
4,LOK,STE20,STE,0.92
5,CHED,CDK,CMGC,0.94
6,CDK8,CDK,CMGC,1.40
7,MAP3K7,STE11,STE,1.50
8,MAP3K7,MLK,TKL,1.50
9,CDK11,CDK,CMGC,1.50


### TAE-684

,kinase.name,kinase.family,kinase.group,bioactivity
0,ROS,Sev,TK,0.49
1,ULK1,ULK,Other,0.83
2,BMPR1B,STKR,TKL,0.85
3,PLK4,PLK,Other,0.93
4,LTK,ALK,TK,0.95
5,FAK,FAK,TK,1.10
6,PYK2,FAK,TK,1.10
7,ALK,ALK,TK,1.10
8,NuaK2,CAMKL,CAMK,1.20
9,LRRK2,LRRK,TKL,1.40


### SKI-606 (PKIDB: Bosulif | Type 1)

,kinase.name,kinase.family,kinase.group,bioactivity
0,ABL1,Abl,TK,0.057
1,KHS1,STE20,STE,0.500
2,LCK,Src,TK,0.590
3,ErbB3,EGFR,TK,0.770
4,SRC,Src,TK,1.000
5,GAK,NAK,Other,1.300
6,FRK,Src,TK,1.400
7,ABL2,Abl,TK,1.500
8,CLIK1,NKF4,Other,2.000
9,GCK,STE20,STE,3.100
